#### Try to improve Random Forest by hypertune


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
real_df = pd.read_csv("Raw_MAIN_DATASET_IMPUTATION_after_deleting_stai_badge_activity_step_goal.csv")
real_df.head(5)


FileNotFoundError: [Errno 2] No such file or directory: 'Raw_MAIN_DATASET_IMPUTATION_after_deleting_stai_badge_activity_step_goal.csv'

#### Import training, testing and validation


In [ ]:
# input
train_inputs = pd.read_csv("train_inputs.csv")
val_inputs = pd.read_csv("val_inputs.csv")
test_inputs = pd.read_csv("test_inputs.csv")
# target column
train_targets = pd.read_csv("train_targets.csv")
val_targets = pd.read_csv("val_targets.csv")
test_targets = pd.read_csv("test_targets.csv")


In [ ]:
print('train_inputs:', train_inputs.shape)
print('train_targets:', train_targets.shape)
print('val_inputs:', val_inputs.shape)
print('val_targets:', val_targets.shape)
print('test_inputs:', test_inputs.shape)
print('test_targets:', test_targets.shape)


train_inputs: (4460, 64)
train_targets: (4460, 1)
val_inputs: (1487, 64)
val_targets: (1487, 1)
test_inputs: (1487, 64)
test_targets: (1487, 1)


#### Hypertunning the Random forest model


<h5 style="color:orange">Import necesary liberies</h5>


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score


<h5 style="color:green">Initialize the randomforest classifier</h4>


In [ ]:
model = RandomForestClassifier(n_jobs=-1, random_state=42)


In [ ]:
# Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30 ,None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}


In [ ]:
# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(train_inputs, train_targets)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
25 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python310\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Python310\lib\site-packages\

RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, None],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   random_state=42, verbose=2)

In [ ]:
best_model = random_search.best_estimator_

# Perform feature selection based on feature importance
selector = SelectFromModel(best_model, threshold='median')
X_train_selected = selector.fit_transform(train_inputs, train_targets)
X_val_selected = selector.transform(val_inputs)
X_test_selected = selector.transform(test_inputs)

# Train the model with selected features
best_model.fit(X_train_selected, train_targets)


c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=20, min_samples_leaf=2, n_jobs=-1,
                       random_state=42)

In [ ]:
train_preds = best_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')


Training Accuracy: 95.70%


In [ ]:
# Evaluate on the validation set
val_preds = best_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')


Validation Accuracy: 78.35%


In [ ]:
# Evaluate on the test set
test_preds = best_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Test Accuracy: 77.67%


<h5 style="color:Red">--------------------------------------------------------------------------------------------------------------------------------------------------------------</h5>


In [ ]:
# Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30 ,40,50,None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}


In [ ]:
# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(train_inputs, train_targets)
best_model = random_search.best_estimator_

# Perform feature selection based on feature importance
selector = SelectFromModel(best_model, threshold='median')
X_train_selected = selector.fit_transform(train_inputs, train_targets)
X_val_selected = selector.transform(val_inputs)
X_test_selected = selector.transform(test_inputs)

# Train the model with selected features
best_model.fit(X_train_selected, train_targets)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
25 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python310\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Python310\lib\site-packages\s

RandomForestClassifier(max_depth=50, min_samples_split=10, n_estimators=200,
                       n_jobs=-1, random_state=42)

In [ ]:
train_preds = best_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Training Accuracy: 94.17%
Validation Accuracy: 79.02%
Test Accuracy: 78.41%


<h5 style="color:Red">--------------------------------------------------------------------------------------------------------------------------------------------------------------</h5>


In [ ]:
# Hyperparameter grid for RandomizedSearchCV
# Expanded Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [300, 400, 500, 600, 700],  # Increased the range of estimators
    'max_depth': [30, 40, 50, 60, 70, None],      # Added more depth levels
    'min_samples_split': [2, 5, 10, 15],                 # Added more granular values
    'min_samples_leaf': [1, 2, 4, 6, 8],                 # Increased min samples per leaf
    'max_features': ['sqrt', 'log2'],                    # Removed 'auto' since it’s invalid
    'bootstrap': [True, False]                           # Added bootstrap as a parameter
}


# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(train_inputs, train_targets)
best_model = random_search.best_estimator_

# Perform feature selection based on feature importance
selector = SelectFromModel(best_model, threshold='median')
X_train_selected = selector.fit_transform(train_inputs, train_targets)
X_val_selected = selector.transform(val_inputs)
X_test_selected = selector.transform(test_inputs)

# Train the model with selected features
best_model.fit(X_train_selected, train_targets)

train_preds = best_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')



Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Accuracy: 94.10%
Validation Accuracy: 79.35%
Test Accuracy: 78.35%


# Random Forest give accuracy 78.75%


In [ ]:
# Hyperparameter grid for RandomizedSearchCV
# Expanded Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [300, 400, 500, 600, 700, 800],
    'max_depth': [30, 40, 50, 60, 70,80, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]                           # Added bootstrap as a parameter
}


# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(train_inputs, train_targets)
best_model = random_search.best_estimator_

# Perform feature selection based on feature importance
selector = SelectFromModel(best_model, threshold='median')
X_train_selected = selector.fit_transform(train_inputs, train_targets)
X_val_selected = selector.transform(val_inputs)
X_test_selected = selector.transform(test_inputs)

# Train the model with selected features
best_model.fit(X_train_selected, train_targets)

train_preds = best_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')



Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Accuracy: 91.52%
Validation Accuracy: 78.95%
Test Accuracy: 78.75%


In [ ]:
# Hyperparameter grid for RandomizedSearchCV
# Expanded Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [300, 400, 500, 600, 700, 800],
    'max_depth': [30, 40, 50, 60, 70,80, 90,100,None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]                           # Added bootstrap as a parameter
}


# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(train_inputs, train_targets)
best_model = random_search.best_estimator_

# Perform feature selection based on feature importance
selector = SelectFromModel(best_model, threshold='median')
X_train_selected = selector.fit_transform(train_inputs, train_targets)
X_val_selected = selector.transform(val_inputs)
X_test_selected = selector.transform(test_inputs)

# Train the model with selected features
best_model.fit(X_train_selected, train_targets)

train_preds = best_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')



Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Accuracy: 91.52%
Validation Accuracy: 78.95%
Test Accuracy: 78.75%


In [ ]:
best_model = RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced')
# Hyperparameter grid for RandomizedSearchCV
# Expanded Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [300, 400, 500, 600, 700, 800],
    'max_depth': [30, 40, 50, 60, 70,80, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]                           # Added bootstrap as a parameter
}


# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(train_inputs, train_targets)
best_model = random_search.best_estimator_

# Perform feature selection based on feature importance
selector = SelectFromModel(best_model, threshold='median')
X_train_selected = selector.fit_transform(train_inputs, train_targets)
X_val_selected = selector.transform(val_inputs)
X_test_selected = selector.transform(test_inputs)

# Train the model with selected features
best_model.fit(X_train_selected, train_targets)

train_preds = best_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')



Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Accuracy: 91.52%
Validation Accuracy: 78.95%
Test Accuracy: 78.75%


In [ ]:
!pip install xgboost


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# Convert train_targets, val_targets, and test_targets to Numpy arrays
train_targets_array = train_targets.to_numpy().ravel()
val_targets_array = val_targets.to_numpy().ravel()
test_targets_array = test_targets.to_numpy().ravel()

# Combine RandomForest and XGBoost in an ensemble model
ensemble_model = VotingClassifier(estimators=[('rf', best_model), ('xgb', xgb_model)], voting='soft', n_jobs=-1)
ensemble_model.fit(X_train_selected, train_targets_array)

# Evaluate the ensemble model on the training set
train_preds = ensemble_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets_array, train_preds)
print(f'Ensemble Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = ensemble_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets_array, val_preds)
print(f'Ensemble Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = ensemble_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets_array, test_preds)
print(f'Ensemble Test Accuracy: {test_accuracy * 100:.2f}%')


Ensemble Training Accuracy: 99.26%
Ensemble Validation Accuracy: 79.09%
Ensemble Test Accuracy: 77.40%


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# Convert targets to Numpy arrays
train_targets_array = train_targets.to_numpy().ravel()
val_targets_array = val_targets.to_numpy().ravel()
test_targets_array = test_targets.to_numpy().ravel()

# Combine RandomForest and XGBoost in an ensemble model with weights
ensemble_model = VotingClassifier(
    estimators=[('rf', best_model), ('xgb', xgb_model)],
    voting='soft',
    weights=[2, 1],  # Weight RandomForest more than XGBoost
    n_jobs=-1
)

# Fit the ensemble model
ensemble_model.fit(X_train_selected, train_targets_array)

# Evaluate the ensemble model
train_preds = ensemble_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets_array, train_preds)
print(f'Ensemble Training Accuracy: {train_accuracy * 100:.2f}%')

val_preds = ensemble_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets_array, val_preds)
print(f'Ensemble Validation Accuracy: {val_accuracy * 100:.2f}%')

test_preds = ensemble_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets_array, test_preds)
print(f'Ensemble Test Accuracy: {test_accuracy * 100:.2f}%')


Ensemble Training Accuracy: 98.97%
Ensemble Validation Accuracy: 79.42%
Ensemble Test Accuracy: 77.54%


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# Convert train_targets, val_targets, and test_targets to Numpy arrays
train_targets_array = train_targets.to_numpy().ravel()
val_targets_array = val_targets.to_numpy().ravel()
test_targets_array = test_targets.to_numpy().ravel()

# Combine RandomForest and XGBoost in an ensemble model
ensemble_model = VotingClassifier(estimators=[('rf', best_model), ('xgb', xgb_model)], voting='soft', n_jobs=-1)
ensemble_model.fit(X_train_selected, train_targets_array)

# Evaluate the ensemble model on the training set
train_preds = ensemble_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets_array, train_preds)
print(f'Ensemble Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = ensemble_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets_array, val_preds)
print(f'Ensemble Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = ensemble_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets_array, test_preds)
print(f'Ensemble Test Accuracy: {test_accuracy * 100:.2f}%')


Ensemble Training Accuracy: 99.26%
Ensemble Validation Accuracy: 79.09%
Ensemble Test Accuracy: 77.40%


# LightGBM

## It give accuracy 76.40%


In [ ]:
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

# Check the class distribution before applying SMOTE
print("Class distribution before SMOTE:", train_targets.value_counts())

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)  # Set k_neighbors to 1 to avoid the error
X_train_resampled, train_targets_resampled = smote.fit_resample(X_train_selected, train_targets_array)

# Check the class distribution after applying SMOTE
print("Class distribution after SMOTE:", pd.Series(train_targets_resampled).value_counts())

# Initialize and fit the LightGBM model
lgb_model = LGBMClassifier(n_estimators=1000, learning_rate=0.05, max_depth=20, random_state=42)
lgb_model.fit(X_train_resampled, train_targets_resampled)

# Evaluate performance
train_preds = lgb_model.predict(X_train_resampled)
train_accuracy = accuracy_score(train_targets_resampled, train_preds)
print(f'Training Accuracy (LightGBM): {train_accuracy * 100:.2f}%')

val_preds = lgb_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets_array, val_preds)
print(f'Validation Accuracy (LightGBM): {val_accuracy * 100:.2f}%')

test_preds = lgb_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets_array, test_preds)
print(f'Test Accuracy (LightGBM): {test_accuracy * 100:.2f}%')


Class distribution before SMOTE: Emotions      
ALERT             3302
HAPPY              321
RESTED/RELAXED     288
NEUTRAL            265
TIRED              125
TENSE/ANXIOUS      117
SAD                 40
ANGER                2
Name: count, dtype: int64
Class distribution after SMOTE: NEUTRAL           3302
ALERT             3302
HAPPY             3302
RESTED/RELAXED    3302
TIRED             3302
TENSE/ANXIOUS     3302
SAD               3302
ANGER             3302
Name: count, dtype: int64
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005016 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8156
[LightGBM] [Info] Number of data points in the train set: 26416, number of used features: 32
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.07

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from collections import Counter


In [ ]:
# Check class distribution before SMOTE
print("Original class distribution:", Counter(train_targets))

# Initialize RandomForestClassifier with class weighting
model = RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced')


Original class distribution: Counter({'Emotions': 1})


In [ ]:
# Expanded Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [300, 400, 500, 600, 700, 800],
    'max_depth': [30, 40, 50, 60, 70, 80, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(train_inputs, train_targets)
best_model = random_search.best_estimator_


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
# Check minimum class size
min_class_size = train_targets.value_counts().min()
k_neighbors = min(6, min_class_size - 1)  # Adjust k_neighbors to be <= min_class_size - 1

# Handle class imbalance using SMOTE with adjusted k_neighbors
smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
X_train_resampled, train_targets_resampled = smote.fit_resample(train_inputs, train_targets)

# Check class distribution after SMOTE
print("Resampled class distribution:", Counter(train_targets_resampled))


Resampled class distribution: Counter({'Emotions': 1})


In [ ]:
# Perform feature selection based on feature importance on the resampled dataset
selector = SelectFromModel(best_model, threshold='median')
X_train_selected = selector.fit_transform(X_train_resampled, train_targets_resampled)
X_val_selected = selector.transform(val_inputs)
X_test_selected = selector.transform(test_inputs)

# Train the model with selected features
best_model.fit(X_train_selected, train_targets_resampled)


c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight='balanced', max_depth=80,
                       min_samples_leaf=2, min_samples_split=10,
                       n_estimators=500, n_jobs=-1, random_state=42)

In [ ]:
# Evaluate on the training set
train_preds = best_model.predict(X_train_selected)
train_accuracy = accuracy_score(train_targets_resampled, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_model.predict(X_val_selected)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_model.predict(X_test_selected)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Training Accuracy: 98.99%
Validation Accuracy: 77.54%
Test Accuracy: 76.73%


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

# Define the model
model = RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced')

# Define the pipeline steps
pipeline = Pipeline(steps=[
    ('smote', SMOTE(random_state=42)),  # Handle class imbalance
    ('scaler', StandardScaler()),        # Scale features
    ('feature_selection', SelectFromModel(estimator=model, threshold='median')),  # Feature selection
    ('classifier', model)                # Classifier
])

# Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'classifier__n_estimators': [300, 400, 500, 600, 700, 800],
    'classifier__max_depth': [30, 40, 50, 60, 70, 80, None],
    'classifier__min_samples_split': [2, 5, 10, 15],
    'classifier__min_samples_leaf': [1, 2, 4, 6, 8],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__bootstrap': [True, False]  # Use both bootstrap options
}

# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(train_inputs, train_targets)

# Best pipeline model
best_pipeline_model = random_search.best_estimator_

# Evaluate on the training set
train_preds = best_pipeline_model.predict(train_inputs)
train_accuracy = accuracy_score(train_targets, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_pipeline_model.predict(val_inputs)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_pipeline_model.predict(test_inputs)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


ValueError: 
All the 100 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python310\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Python310\lib\site-packages\sklearn\pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
  File "c:\Python310\lib\site-packages\sklearn\pipeline.py", line 386, in _fit
    self._validate_steps()
  File "c:\Python310\lib\site-packages\sklearn\pipeline.py", line 256, in _validate_steps
    raise TypeError(
TypeError: All intermed

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from collections import Counter

# Check original class distribution
print("Original class distribution:", Counter(train_targets))

# Handle class imbalance using SMOTE before pipeline
smote = SMOTE(random_state=42)
X_train_resampled, train_targets_resampled = smote.fit_resample(train_inputs, train_targets)

# Check resampled class distribution
print("Resampled class distribution:", Counter(train_targets_resampled))

# Define the model
model = RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced')

# Define the pipeline steps
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),  # Scale features
    ('feature_selection', SelectFromModel(estimator=model, threshold='median')),  # Feature selection
    ('classifier', model)  # Classifier
])

# Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'classifier__n_estimators': [300, 400, 500, 600, 700, 800],
    'classifier__max_depth': [30, 40, 50, 60, 70, 80, None],
    'classifier__min_samples_split': [2, 5, 10, 15],
    'classifier__min_samples_leaf': [1, 2, 4, 6, 8],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__bootstrap': [True, False]
}

# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(X_train_resampled, train_targets_resampled)

# Best pipeline model
best_pipeline_model = random_search.best_estimator_

# Evaluate on the training set
train_preds = best_pipeline_model.predict(X_train_resampled)
train_accuracy = accuracy_score(train_targets_resampled, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_pipeline_model.predict(val_inputs)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_pipeline_model.predict(test_inputs)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Original class distribution: Counter({'Emotions': 1})


ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 2, n_samples = 2

Original class distribution: Counter({'Emotions': 1})


ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 3, n_samples_fit = 2, n_samples = 2

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from collections import Counter

# Check original class distribution
print("Original class distribution:", Counter(train_targets))

# Handle class imbalance using SMOTE before pipeline
smote = SMOTE(random_state=42, k_neighbors=2)  # Adjust k_neighbors if necessary
X_train_resampled, train_targets_resampled = smote.fit_resample(train_inputs, train_targets)

# Check resampled class distribution
print("Resampled class distribution:", Counter(train_targets_resampled))

# Define the model
model = RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced')

# Define the pipeline steps
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),  # Scale features
    ('feature_selection', SelectFromModel(estimator=model, threshold='median')),  # Feature selection
    ('classifier', model)  # Classifier
])

# Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'classifier__n_estimators': [300, 400, 500, 600, 700, 800],
    'classifier__max_depth': [30, 40, 50, 60, 70, 80, None],
    'classifier__min_samples_split': [2, 5, 10, 15],
    'classifier__min_samples_leaf': [1, 2, 4, 6, 8],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__bootstrap': [True, False]
}

# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(X_train_resampled, train_targets_resampled)

# Best pipeline model
best_pipeline_model = random_search.best_estimator_

# Evaluate on the training set
train_preds = best_pipeline_model.predict(X_train_resampled)
train_accuracy = accuracy_score(train_targets_resampled, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_pipeline_model.predict(val_inputs)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_pipeline_model.predict(test_inputs)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Original class distribution: Counter({'Emotions': 1})


ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 3, n_samples_fit = 2, n_samples = 2

In [ ]:
print(f"Train resampled shape: {X_train_resampled.shape}, {len(train_targets_resampled)}")
print(f"Validation shape: {val_inputs.shape}, {len(val_targets)}")
print(f"Test shape: {test_inputs.shape}, {len(test_targets)}")


Train resampled shape: (26416, 64), 26416
Validation shape: (1487, 64), 1487
Test shape: (1487, 64), 1487


==================================================================================================================================================================================


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from collections import Counter


In [ ]:
# Check original class distribution
print("Original class distribution:", Counter(train_targets))


Original class distribution: Counter({'Emotions': 1})


In [ ]:
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

# Check original class distribution
print("Original class distribution:", Counter(train_targets))

# Check the minimum number of samples in any class
min_class_size = min(Counter(train_targets).values())
print(f'Minimum class size: {min_class_size}')

# Ensure k_neighbors is set to at least 1
k_neighbors = max(1, min(2, min_class_size - 1))  # Ensure k_neighbors is <= min_class_size - 1
print(f'Using k_neighbors = {k_neighbors}')

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
X_train_resampled, train_targets_resampled = smote.fit_resample(train_inputs, train_targets)

# Check resampled class distribution
print("Resampled class distribution:", Counter(train_targets_resampled))

# Define the model
model = RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced')

# Define the pipeline steps
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),  # Scale features
    ('feature_selection', SelectFromModel(estimator=model, threshold='median')),  # Feature selection
    ('classifier', model)  # Classifier
])

# Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'classifier__n_estimators': [300, 400, 500, 600, 700, 800],
    'classifier__max_depth': [30, 40, 50, 60, 70, 80, None],
    'classifier__min_samples_split': [2, 5, 10, 15],
    'classifier__min_samples_leaf': [1, 2, 4, 6, 8],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__bootstrap': [True, False]
}

# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=20, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(X_train_resampled, train_targets_resampled)

# Best pipeline model
best_pipeline_model = random_search.best_estimator_

# Evaluate on the training set
train_preds = best_pipeline_model.predict(X_train_resampled)
train_accuracy = accuracy_score(train_targets_resampled, train_preds)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate on the validation set
val_preds = best_pipeline_model.predict(val_inputs)
val_accuracy = accuracy_score(val_targets, val_preds)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Evaluate on the test set
test_preds = best_pipeline_model.predict(test_inputs)
test_accuracy = accuracy_score(test_targets, test_preds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Original class distribution: Counter({'Emotions': 1})
Minimum class size: 1
Using k_neighbors = 1
Resampled class distribution: Counter({'Emotions': 1})
Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Accuracy: 99.36%
Validation Accuracy: 77.87%
Test Accuracy: 76.06%
